<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Supercharge_RAG_With_Contextualized_Late_Interactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required dependencies

In [ ]:
!pip install -q llama-hub
!pip install -q arxiv
!pip install -q semanticscholar
!pip install -q sentence-transformers==2.3.0
!pip install -q ragatouille
!pip install -q llama-index
!pip install -q llama-index-readers-file
!pip intall -q llama-index-llms-openai
!pip install -q llama-index-core
!pip install llama-index-embeddings-huggingface
!pip install llama-index-vector-stores-faiss
!pip install -q langchain
!pip install -q langchain-core
!pip intall -q langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s e

```
linkify-it-py==2.0.3
llama-hub==0.0.79.post1
llama-index==0.10.30
llama-index-agent-openai==0.2.2
llama-index-cli==0.1.12
llama-index-core==0.10.30
llama-index-embeddings-huggingface==0.2.0
llama-index-embeddings-openai==0.1.8
llama-index-indices-managed-llama-cloud==0.1.5
llama-index-legacy==0.9.48
llama-index-llms-openai==0.1.16
llama-index-multi-modal-llms-openai==0.1.5
llama-index-program-openai==0.1.5
llama-index-question-gen-openai==0.1.3
llama-index-readers-file==0.1.19
llama-index-readers-llama-parse==0.1.4
llama-index-vector-stores-faiss==0.1.2
llama-parse==0.4.1
RAGatouille==0.0.8.post2
```

## Load the Dataset

In [ ]:
!wget https://arxiv.org/pdf/2306.02707.pdf

--2024-04-28 14:41:07--  https://arxiv.org/pdf/2306.02707.pdf
Resolving arxiv.org (arxiv.org)... 151.101.67.42, 151.101.131.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/2306.02707 [following]
--2024-04-28 14:41:08--  http://arxiv.org/pdf/2306.02707
Connecting to arxiv.org (arxiv.org)|151.101.67.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1458242 (1.4M) [application/pdf]
Saving to: ‘2306.02707.pdf’

2306.02707.pdf      100%[===================>]   1.39M  --.-KB/s    in 0.07s   

2024-04-28 14:41:08 (20.3 MB/s) - ‘2306.02707.pdf’ saved [1458242/1458242]



In [ ]:
from llama_index.readers.file import PDFReader
loader = PDFReader()
documents = loader.load_data("2306.02707.pdf")

In [ ]:
print(len(documents))

51


In [ ]:
list_documents = [ document.text for document in documents ]

## Opensource Embedding Model

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore

In [ ]:
index = VectorStoreIndex.from_documents(documents)

## Query Index

In [ ]:
retriver = index.as_retriever(similarity_top_k=3)

In [ ]:
similar_docs = retriver.retrieve("What is instruction tuning?")

In [ ]:
similar_docs1 = retriver.retrieve("What is orca?")

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
for node in similar_docs1:
  display_source_node(node)

**Node ID:** 144c4c2e-681f-444d-bd3f-98b745b28c06<br>**Similarity:** 0.6652078795266653<br>**Text:** For Vicuna and Orca the format of the prompt is as follows:
### System:
### Human:
Q: Which blood...<br>

**Node ID:** 98e45e23-9d19-4e6a-bc0e-adcd2561e554<br>**Similarity:** 0.6508155769295554<br>**Text:** Task / System Message Empty Follow Well Detailed Answer
AQuA-RAT 27.9 21.3 25.2
LogiQA 35.2 36.4 ...<br>

**Node ID:** cbd45178-6472-4bbe-b435-78ebf64e2ad3<br>**Similarity:** 0.6503899050959893<br>**Text:** 0102030405060708090AQuA-RAT (GRE Math, GMAT Math)
LogiQA (English, Civil Services)
LSAT-AR
LSAT-L...<br>

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
for node in similar_docs:
  display_source_node(node)


**Node ID:** 61299ed3-a6eb-4f9f-917a-9f731f154305<br>**Similarity:** 0.7422756912118058<br>**Text:** Model Tuning Method Data Size Teacher
Alpaca Simple Instructions / Self-instruct 52K text-da-vinc...<br>

**Node ID:** 074197f9-75d9-48e4-91a6-59665ae927d8<br>**Similarity:** 0.7226097983044915<br>**Text:** Contents
1 Introduction 4
1.1 Challenges with Existing Methods . . . . . . . . . . . . . . . . . ...<br>

**Node ID:** 6469e949-d4d8-40f4-be8f-836abf0e44c5<br>**Similarity:** 0.703846265300339<br>**Text:** content generation and information-seeking queries over other types of tasks. Therefore,
models t...<br>

In [ ]:
!mkdir open_source
# save index to disk
index.storage_context.persist("/content/open_source")

In [ ]:
for i,node in enumerate(similar_docs):
  print(f"------------------ {i} ----------------------------------------------------")
  print(node.text)
  print("----------------------------------------------------------------------------")

------------------ 0 ----------------------------------------------------
Model Tuning Method Data Size Teacher
Alpaca Simple Instructions / Self-instruct 52K text-da-vinci-003
Vicuna User Instructions / Natural 70K ChatGPT
Dolly User Instructions / Natural 15K Human
WizardLM Complex Instructions / Evol-instruct 250K ChatGPT
Orca Complex Instructions / Explanations 5M ChatGPT (5M)
∩GPT-4 (1M)
Table 1: Overview of popular models instruction tuned with OpenAI large foundation models
(LFMs). Orca leverages complex instructions and explanations for progressive learning.
User Instruction: Use the given data to calculate the median. 
Input:[7, 3, 8, 2, 10]
User Instruction: Answer this question.
Input:Which small lake lies between Windermere and Grasmere?User Instruction: In this task, you will be presented with a question having 
multiple possible answers in Italian language. And you should choose a most 
suitable option out of "A", "B", "C", "D", and "E" based on your commonsense 
knowledg

## Using OpenAI embeddings

In [ ]:
# get API key and create embeddings
import os
import openai
from google.colab import userdata


openai.api_key = userdata.get("OPENAI_API_KEY")
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import Settings

# global default
Settings.embed_model = OpenAIEmbedding()

In [ ]:
index = VectorStoreIndex.from_documents(documents)

In [ ]:
retriver_openai = index.as_retriever(similarity_top_k=3)

In [ ]:
similar_docs = retriver_openai.retrieve("What is instruction tuning?")

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
for node in similar_docs:
  display_source_node(node)

**Node ID:** 2ab26823-162c-4c25-a57f-480b05f87a04<br>**Similarity:** 0.8330309385389002<br>**Text:** Model Tuning Method Data Size Teacher
Alpaca Simple Instructions / Self-instruct 52K text-da-vinc...<br>

**Node ID:** fced3be6-00b7-4072-9dca-3faef2a3fe19<br>**Similarity:** 0.8035424483865514<br>**Text:** System instructions are sampled from a diverse instruction set including chain-of-thought
reasoni...<br>

**Node ID:** 857f4aec-960f-4b34-ae86-74b218a94283<br>**Similarity:** 0.7809231596633633<br>**Text:** System Instruction: You are an AI assistant. User will you give you a task. Your 
goal is to comp...<br>

In [ ]:
for i,node in enumerate(similar_docs):
  print(f"------------------ {i} ----------------------------------------------------")
  print(node.text)
  print("----------------------------------------------------------------------------")

------------------ 0 ----------------------------------------------------
Model Tuning Method Data Size Teacher
Alpaca Simple Instructions / Self-instruct 52K text-da-vinci-003
Vicuna User Instructions / Natural 70K ChatGPT
Dolly User Instructions / Natural 15K Human
WizardLM Complex Instructions / Evol-instruct 250K ChatGPT
Orca Complex Instructions / Explanations 5M ChatGPT (5M)
∩GPT-4 (1M)
Table 1: Overview of popular models instruction tuned with OpenAI large foundation models
(LFMs). Orca leverages complex instructions and explanations for progressive learning.
User Instruction: Use the given data to calculate the median. 
Input:[7, 3, 8, 2, 10]
User Instruction: Answer this question.
Input:Which small lake lies between Windermere and Grasmere?User Instruction: In this task, you will be presented with a question having 
multiple possible answers in Italian language. And you should choose a most 
suitable option out of "A", "B", "C", "D", and "E" based on your commonsense 
knowledg

In [ ]:
similar_docs1 = retriver.retrieve("What is orca?")
from llama_index.core.response.notebook_utils import display_source_node
for node in similar_docs1:
  display_source_node(node)

**Node ID:** 144c4c2e-681f-444d-bd3f-98b745b28c06<br>**Similarity:** 0.6652078795266653<br>**Text:** For Vicuna and Orca the format of the prompt is as follows:
### System:
### Human:
Q: Which blood...<br>

**Node ID:** 98e45e23-9d19-4e6a-bc0e-adcd2561e554<br>**Similarity:** 0.6508155769295554<br>**Text:** Task / System Message Empty Follow Well Detailed Answer
AQuA-RAT 27.9 21.3 25.2
LogiQA 35.2 36.4 ...<br>

**Node ID:** cbd45178-6472-4bbe-b435-78ebf64e2ad3<br>**Similarity:** 0.6503899050959893<br>**Text:** 0102030405060708090AQuA-RAT (GRE Math, GMAT Math)
LogiQA (English, Civil Services)
LSAT-AR
LSAT-L...<br>

In [ ]:
for i,node in enumerate(similar_docs1):
  print(f"------------------ {i} ----------------------------------------------------")
  print(node.text)
  print("----------------------------------------------------------------------------")

------------------ 0 ----------------------------------------------------
For Vicuna and Orca the format of the prompt is as follows:
### System:
### Human:
Q: Which blood type makes the best CEOs? Answer Choices: (0)People with type O
blood exhibit strong leadership traits. (1)Blood type has no impact on someone’s
abilities. (2)People with type O blood make the best leaders. (3)The best CEOs have
type O blood. A: Among 0 through 3, the answer is
### Assistant:
Figure 13: Prompt format for Vicuna and Orca used in all multiple choice-question answering
tasks including TruthfulQA-MC and those in AGIEval and Big-Bench Hard.
Evaluation results: From Figure 14, we observe that Orca’s outputs are significantly more
truthful than Vicuna-13B; while trailing behind ChatGPT and GPT-4.
Instruction following results: Vicuna output format is quite noisy in general – as it often
does not follow the prompt template – compared to Orca, ChatGPT and GPT-4 which took
us significantly more time to manuall

## Ragatouille

In [ ]:
from llama_index.core.llama_pack import download_llama_pack
RAGatouille_retriever = download_llama_pack("RAGatouilleRetrieverPack","./ragatouille_pack")

In [ ]:
ragatouille_pack = RAGatouille_retriever(documents,
                                         index_name="orca",
                                         top_k=3,)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

...

[Apr 21, 08:55:41] [0] 		 #> Encoding 219 passages..



100%|██████████| 7/7 [03:26<00:00, 29.53s/it]
1it [03:28, 208.76s/it]
100%|██████████| 1/1 [00:00<00:00, 868.57it/s]

[Apr 21, 08:59:09] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 21, 08:59:09] #> Building the emb2pid mapping..
[Apr 21, 08:59:09] len(emb2pid) = 34105



100%|██████████| 2048/2048 [00:00<00:00, 46745.40it/s]

[Apr 21, 08:59:10] #> Saved optimized IVF to .ragatouille/colbert/indexes/orca/ivf.pid.pt
Done indexing!
LLM is explicitly disabled. Using MockLLM.


In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

retriever = ragatouille_pack.get_modules()["retriever"]
nodes = retriever.retrieve("What is instruction tuning?")

for node in nodes:
    display_source_node(node)

Loading searcher for index orca for the first time... This may take a few seconds
[Apr 21, 08:59:22] #> Loading codec...
[Apr 21, 08:59:22] #> Loading IVF...
[Apr 21, 08:59:22] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


[Apr 21, 08:59:53] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 3591.01it/s]

[Apr 21, 08:59:53] #> Loading codes and residuals...


...

[Apr 21, 09:00:24] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is instruction tuning?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  7899, 17372,  1029,   102,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


**Node ID:** 6f94ba79-81cd-47a4-b777-ab01921fa54f<br>**Similarity:** 26.51420021057129<br>**Text:** Given user instructions for a task and an input,
the system generates a response. Existing works ...<br>

**Node ID:** bb290fd0-7937-46c2-bd9a-fed687471792<br>**Similarity:** 26.102283477783203<br>**Text:** For multimodal tasks, instruction tuning has been used to generate
synthetic instruction-followin...<br>

**Node ID:** 9e168d49-957f-42b4-853c-72b1ed984e25<br>**Similarity:** 21.646852493286133<br>**Text:** For example, we notice
that models that are instruction-tuned with GPT-4 responses tend to genera...<br>

In [ ]:
for i,node in enumerate(nodes):
  print(f"------------------ {i} ----------------------------------------------------")
  print(node.text)
  print("----------------------------------------------------------------------------")

------------------ 0 ----------------------------------------------------
Given user instructions for a task and an input,
the system generates a response. Existing works like Alpaca [ 7], Vicuna [ 9] and variants
follow a similar template to train small models with ⟨{user instruction, input}, output ⟩.
2 Preliminaries
2.1 Instruction Tuning
Instruction tuning [ 22] is a technique that allows pre-trained language models to learn
from input (natural language descriptions of the task) and response pairs, for example,
{"instruction": "Arrange the words in the given sentence to form a grammatically
correct sentence.", "input": "the quickly brown fox jumped", "output": "the brown
fox jumped quickly"} . Instruction tuning has been applied to both language-only and
multimodal tasks. For language-only tasks, instruction tuning has been shown to improve
the zero-shot and few-shot performance of models such as FLAN [ 22] and InstructGPT [ 5]
on various benchmarks. For multimodal tasks, instructi

## Implementing ColBert In Langchain

In [ ]:
from ragatouille import RAGPretrainedModel
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[Apr 28, 14:44:56] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


## Load the Document

In [ ]:
loaders = PyPDFLoader("/content/2306.02707.pdf")
pages = loaders.load()
print(len(pages))

51


In [ ]:
full_document = ""
for page in pages:
  full_document += page.page_content

## Build Index

In [ ]:
RAG.index(collection=[full_document],
          index_name="orca_paper",
          max_document_length=512,
          split_documents=True,
          use_faiss=True
          )

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Apr 28, 14:49:23] #> Creating directory .ragatouille/colbert/indexes/orca_paper 


[Apr 28, 14:49:26] [0] 		 #> Encoding 87 passages..


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
 33%|███▎      | 1/3 [01:00<02:01, 60.61s/it]/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 3/3 [02:46<00:00, 55.55s/it]

[Apr 28, 14:52:12] [0] 		 avg_doclen_est = 357.8735656738281 	 len(local_sample) = 87
[Apr 28, 14:52:12] [0] 		 Creating 2,048 partitions.
[Apr 28, 14:52:12] [0] 		 *Estimated* 31,135 embeddings.
[Apr 28, 14:52:12] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/orca_paper/plan.json ..


used 18 iterations (26.6772s) to cluster 29579 items into 2048 clusters
[0.032, 0.035, 0.029, 0.029, 0.029, 0.032, 0.032, 0.029, 0.03, 0.032, 0.03, 0.031, 0.031, 0.032, 0.031, 0.032, 0.029, 0.032, 0.03, 0.032, 0.03, 0.032, 0.031, 0.031, 0.029, 0.03, 0.032, 0.033, 0.031, 0.034, 0.029, 0.033, 0.032, 0.03, 0.031, 0.029, 0.035, 0.032, 0.031, 0.036, 0.033, 0.032, 0.03, 0.031, 0.03, 0.029, 0.031, 0.035, 0.032, 0.032, 0.03, 0.031, 0.032, 0.03, 0.031, 0.032, 0.033, 0.031, 0.035, 0.031, 0.032, 0.033, 0.032, 0.032, 0.033, 0.032, 0.029, 0.031, 0.028, 0.03, 0.033, 0.03, 0.031, 0.032, 0.032, 0.032, 0.033, 0.034, 0.032, 0.035, 0.034, 0.033, 0.03, 0.034, 0.029, 0.03, 0.032, 0.033, 0.029, 0.036, 0.03, 0.033, 0.031, 0.03, 0.031, 0.031, 0.032, 0.03, 0.03, 0.03, 0.031, 0.034, 0.03, 0.031, 0.031, 0.027, 0.031, 0.028, 0.03, 0.029, 0.032, 0.033, 0.033, 0.029, 0.032, 0.031, 0.032, 0.031, 0.03, 0.032, 0.03, 0.03, 0.032, 0.033, 0.031, 0.033, 0.03, 0.03]


0it [00:00, ?it/s]

...


100%|██████████| 3/3 [02:33<00:00, 51.11s/it]
1it [02:34, 154.76s/it]
100%|██████████| 1/1 [00:00<00:00, 752.34it/s]

[Apr 28, 14:55:14] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 28, 14:55:14] #> Building the emb2pid mapping..
[Apr 28, 14:55:14] len(emb2pid) = 31135



100%|██████████| 2048/2048 [00:00<00:00, 60647.53it/s]

[Apr 28, 14:55:14] #> Saved optimized IVF to .ragatouille/colbert/indexes/orca_paper/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/orca_paper'

## Ask Query

In [ ]:
results = RAG.search(query="What is instruction fine tuning?")
results

Loading searcher for index orca_paper for the first time... This may take a few seconds
[Apr 28, 14:56:05] #> Loading codec...
[Apr 28, 14:56:05] #> Loading IVF...
[Apr 28, 14:56:05] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


[Apr 28, 14:56:32] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 2671.53it/s]

[Apr 28, 14:56:32] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 125.81it/s]

[Apr 28, 14:56:32] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Apr 28, 14:56:57] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What is instruction fine tuning?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003,  7899,  2986, 17372,  1029,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'content': 'And you should choose a most \nsuitable option out of "A", "B", "C", "D", and "E" based on your commonsense \nknowledge. \nInput:Solve this question: Dove non riusciresti a vedere la luce? \nOptions: A scrivaniaB frigoriferoC sole D universoE atticoOutput: First, we need to arrange the data in ascending order: [2, 3, 7, 8, 10]. \nSince there are 5 numbers, the median is the middle number, which is 7.\nOutput: B frigorifero\nOutput: Rydal Water lies between Windermere and Grasmere.\nFigure 4: Instruction-tuning with GPT-49. Given user instructions for a task and an input,\nthe system generates a response. Existing works like Alpaca [ 7], Vicuna [ 9] and variants\nfollow a similar template to train small models with ⟨{user instruction, input}, output ⟩.\n2 Preliminaries\n2.1 Instruction Tuning\nInstruction tuning [ 22] is a technique that allows pre-trained language models to learn\nfrom input (natural language descriptions of the task) and response pairs, for example,\n{"i

In [ ]:
results = RAG.search(query="What is instruction SQUAD V2.0?",k=2)
results

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'content': '. . . . . . . . . . . . . . . . . . . . . . . . . . . . . 9\n3.1.2 Dataset Description and Sampling from the FLAN-v2 Collection . . . 9\n3.1.3 ChatGPT as Teaching Assistant . . . . . . . . . . . . . . . . . . . . . 12\n3.2 Training . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 13\n4 Experiment Setup 14\n4.1 Baselines . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 14\n4.2 Tasks . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 15\n4.2.1 Open-ended Generation Capabilities . . . . . . . . . . . . . . . . . . . 15\n4.2.2 Reasoning Capabilities . . . . . . . . . . . . . . . . . . . . . . . . . .',
  'score': 14.657217979431152,
  'rank': 1,
  'document_id': 'd6c007be-a8dc-4c41-a038-3fd333ca334c',
  'passage_id': 2},
 {'content': 'A wide range of works in recent times, including Alpaca [ 7], Vicuna [ 9], WizardLM [ 8] and\nKoala [14], have adopted instruction-tuning to train smaller lan

## Use as a langchai Retriever to retrieve context

In [ ]:
retriver = RAG.as_langchain_retriever(k=3)
retriver.invoke("What is instruction fine tuning?")

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[Document(page_content='And you should choose a most \nsuitable option out of "A", "B", "C", "D", and "E" based on your commonsense \nknowledge. \nInput:Solve this question: Dove non riusciresti a vedere la luce? \nOptions: A scrivaniaB frigoriferoC sole D universoE atticoOutput: First, we need to arrange the data in ascending order: [2, 3, 7, 8, 10]. \nSince there are 5 numbers, the median is the middle number, which is 7.\nOutput: B frigorifero\nOutput: Rydal Water lies between Windermere and Grasmere.\nFigure 4: Instruction-tuning with GPT-49. Given user instructions for a task and an input,\nthe system generates a response. Existing works like Alpaca [ 7], Vicuna [ 9] and variants\nfollow a similar template to train small models with ⟨{user instruction, input}, output ⟩.\n2 Preliminaries\n2.1 Instruction Tuning\nInstruction tuning [ 22] is a technique that allows pre-trained language models to learn\nfrom input (natural language descriptions of the task) and response pairs, for exa